# CHARIS MCMC

To speed things up, we use jax functions.

In [1]:
import multiprocessing as mp
import os
mp.set_start_method("spawn", force=True) # Jax was slowing down from os.fork() and this fixed it
os.environ["JAX_PLATFORM_NAME"] = "cpu" # Jax wasn't working with our GPU for unknown reasons
import numpy as np
from pathlib import Path
from pyPolCal.csv_tools import read_csv_physical_model_all_bins,arr_csv_HWP
from pyPolCal.instruments_jax import *
from pyPolCal.utils import generate_system_mueller_matrix
from pyMuellerMat.physical_models.charis_physical_models import HWP_retardance,IMR_retardance

/opt/anaconda3/envs/charisenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Step 1, Read in Data

First, I'm going to read the CSVs for all wavelength bins at once so we can do a global fit. There are some configurations for `read_csv_physical_model_all_bins()` that are detailed in the docstring, but for a standard internal calibration just leave the `mode` empty.

In [2]:
csv_dir = Path('datacsvs/csvs_pickoff')
interleaved_values_all, interleaved_stds_all, configuration_list_all = read_csv_physical_model_all_bins(csv_dir)
configuration_list_all


[{'hwp': {'theta': 0.0, 'wavelength': 1159.5614},
  'image_rotator': {'theta': 45.0, 'wavelength': 1159.5614},
  'wollaston': {'wavelength': 1159.5614}},
 {'hwp': {'theta': 45.0, 'wavelength': 1159.5614},
  'image_rotator': {'theta': 45.0, 'wavelength': 1159.5614},
  'wollaston': {'wavelength': 1159.5614}},
 {'hwp': {'theta': 11.25, 'wavelength': 1159.5614},
  'image_rotator': {'theta': 45.0, 'wavelength': 1159.5614},
  'wollaston': {'wavelength': 1159.5614}},
 {'hwp': {'theta': 56.25, 'wavelength': 1159.5614},
  'image_rotator': {'theta': 45.0, 'wavelength': 1159.5614},
  'wollaston': {'wavelength': 1159.5614}},
 {'hwp': {'theta': 22.5, 'wavelength': 1159.5614},
  'image_rotator': {'theta': 45.0, 'wavelength': 1159.5614},
  'wollaston': {'wavelength': 1159.5614}},
 {'hwp': {'theta': 67.5, 'wavelength': 1159.5614},
  'image_rotator': {'theta': 45.0, 'wavelength': 1159.5614},
  'wollaston': {'wavelength': 1159.5614}},
 {'hwp': {'theta': 33.75, 'wavelength': 1159.5614},
  'image_rotator'

# Step 2: Ready Model For MCMC

Now, I'm going to set up the model for MCMC. The initial setup is identical to the plotting setup using in `CHARIS_minimize_internalcal_example.ipynb`. Refer to that notebook for more detailed information about the `system_dict`, `p0_dict`, and the model generation functions. For MCMC, there are a couple additional steps. 
`bounds` is a bounds dictionary. It has the same format as the `p0_dict`, but instead of initial guesses put in in bounds for each component. The `prior_dict` has a similar setup, but for each component you need a `type` with corresponding `kwargs`. The format for uniform and Gaussian priors is below. Finally, define an `output_h5` path. This will be the backend for MCMC. It will store your emcee chains and is used in the plotting functions.

In [ ]:
# Getting the system dictionary setup and defining starting guesses values
wavelength_bins = np.array([1159.5614, 1199.6971, 1241.2219, 1284.184 , 1328.6331, 1374.6208,
1422.2002, 1471.4264, 1522.3565, 1575.0495, 1629.5663, 1685.9701,
1744.3261, 1804.7021, 1867.1678, 1931.7956, 1998.6603, 2067.8395,
2139.4131, 2213.4641, 2290.0781, 2369.3441])
wavelength_bin = 15 # placeholder
epsilon_cal = 1 # defining as perfect, reasoning in Joost t Hart 2021
offset_imr = 0 
offset_hwp = 0
offset_cal = 0
imr_theta = 0 # placeholder 
hwp_theta = 0 # placeholder
# Past fits from scipy minimize on the naive fits
d = 259.7 
wsio2 = 1.617
wmgf2 = 1.264

# Define instrument configuration as system dictionary
# Wollaston beam, imr theta/phi, and hwp theta/phi will all be updated within functions, so don't worry about their values here

system_dict = {
"components" : {
    "wollaston" : {
        "type" : "CHARIS_wollaston_function",
        "properties" : {"beam": 'o'}, 
        "tag": "internal",
    },
    "image_rotator" : {
        "type" : "elliptical_IMR_function",
        "properties" : {"wavelength":wavelength_bins[wavelength_bin]},
        "tag": "internal",
    },
    "hwp" : {
        "type" : "two_layer_HWP_function",
        "properties" : {"wavelength": wavelength_bins[wavelength_bin], "w_SiO2": wsio2, "w_MgF2": wmgf2, "theta":hwp_theta, "delta_theta": offset_hwp},
        "tag": "internal",
    },
    "lprot": { # changed from delta_theta to match Joost t Hart
        "type": "rotator_function",
        "properties" : {'pa':offset_cal},
        "tag": "internal",
    },
    "lp" : {  # calibration polarizer for internal calibration source
        "type": "diattenuator_retarder_function",
        "properties": {"epsilon": epsilon_cal},
        "tag": "internal",
    }}
}
    
# Starting guesses

p0_dict = {
    "image_rotator" : 
        {"delta_theta": offset_imr},
    "hwp" :  
        {"w_SiO2": wsio2, "w_MgF2": wmgf2,"delta_theta": offset_hwp},
    "lprot" : 
        {"pa": offset_cal}
}

system_mm = generate_system_mueller_matrix(system_dict) # Generating pyMuellerMat system MM

p0 = [1.623, 1.268, 262.56] # Starting guesses from Joost t Hart 2021 
offset_bounds = (-5.0,5.0) # Wider bounds than first MCMC run, offsets were converging to their bounds at first
d_bounds = (0.8*p0[2], 1.2*p0[2]) # Physical parameters shouldn't have changed much
imr_offset_bounds = offset_bounds
wsio2_bounds = (0.8*p0[0], 1.2*p0[0])
wmgf2_bounds = (0.8*p0[1], 1.2*p0[1])
hwp_offset_bounds = offset_bounds
cal_offset_bounds = offset_bounds

bounds = {
    "image_rotator" : 
        {"delta_theta": offset_bounds},
    "hwp" :  
        {"w_SiO2": wsio2_bounds, "w_MgF2": wmgf2_bounds, "delta_theta": offset_bounds},
    "lprot" : 
        {"pa": offset_bounds}
}

# define priors
prior_dict = {
    "image_rotator": {
        "delta_theta": {"type": "uniform", "kwargs": {"low":-5, "high": 5}},
    },
    "hwp": {
        "w_SiO2": {"type": "uniform", "kwargs": {"low": 0.8*p0[0], "high": 1.2*p0[0]}},
        "w_MgF2":{"type": "uniform", "kwargs": {"low": 0.8*p0[1], "high": 1.2*p0[1]}},
        "delta_theta": {"type": "uniform", "kwargs": {"low":-5, "high": 5}},
    },
    "lprot": {
        "pa": {"type": "uniform", "kwargs": {"low":-5, "high": 5}},
}}

# backend h5
output_h5 = Path('mcmc_output_dds_naive_imr_uniform_priors.h5')

# Step 3 (Optional): Run Scipy Minimize Globally

Since getting the code ready for Scipy minimize and MCMC is identical, you can run minimize if you would like to get a good idea of fits. Or if you are happy with these, you can skip MCMC entirely. There are two modes for the `minimize_system_mueller_matrix()` function, one uses scipy `minimize` and the other uses scipy `least_squares`. You can choose to include or not include errors. Always set `include_sums` to be false. The modeled double sums are nonsense for CHARIS.

In [6]:
# Minimize everything globally with least squares
    # Counters for iterative fitting
from pyPolCal.fitting import minimize_system_mueller_matrix

iteration = 1
previous_logl = 1000000
new_logl = 0
boundslist = [offset_bounds,wsio2_bounds,wmgf2_bounds,offset_bounds,offset_bounds]
interleaved_difs = interleaved_values_all
difer = interleaved_stds_all

    # Perform iterative fitting
    # MODIFY THE BOUNDS INPUT HERE IF YOU WANT TO CHANGE PARAMETERS
while abs(previous_logl - new_logl) > 0.01*abs(previous_logl):
    if iteration > 1:
        previous_logl = new_logl
    # Configuring minimization function for CHARIS
    result, new_logl,err = minimize_system_mueller_matrix(p0_dict, system_mm, interleaved_difs, 
         configuration_list_all,s_in=[1,0,0,0], process_dataset=process_dataset,process_model=process_model,process_errors=process_errors,bounds = boundslist,include_sums=False,mode='least_squares')
    print(result)

    # Update p0 with new values

    update_p0(p0_dict, result.x)
    iteration += 1

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.4537e+00                                    1.34e+00    
       1              2         1.4345e+00      1.92e-02       1.49e-01       4.97e-02    
       2              3         1.4342e+00      2.81e-04       3.59e-01       3.55e-02    
       3              4         1.4340e+00      2.28e-04       6.75e-01       2.47e-02    
       4              5         1.4338e+00      1.79e-04       6.89e-01       1.69e-02    
       5              6         1.4337e+00      1.08e-04       5.44e-01       1.28e-02    
       6              7         1.4336e+00      9.33e-05       5.34e-01       9.21e-03    
       7              8         1.4335e+00      1.12e-04       3.61e-01       4.40e-03    
       8              9         1.4335e+00      3.98e-05       6.88e-01       2.69e-03    
       9             10         1.4334e+00      2.14e-05       4.06e-01       1.51e-03    

/home/thomasmc/pyPolCal/pyPolCal/fitting.py:164: RuntimeWarning: invalid value encountered in sqrt
  errors = np.sqrt(np.diag(cov))


   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.4334e+00                                    8.58e-05    
       1              2         1.4334e+00      1.86e-08       6.68e-01       5.77e-05    
       2              3         1.4334e+00      8.34e-09       3.77e-01       3.77e-05    
`ftol` termination condition is satisfied.
Function evaluations 3, initial cost 1.4334e+00, final cost 1.4334e+00, first-order optimality 3.77e-05.
     message: `ftol` termination condition is satisfied.
     success: True
      status: 2
         fun: [ 3.844e-02  1.291e-01 ...  1.661e-02 -1.268e-01]
           x: [-1.573e-01  1.750e+00  1.522e+00 -2.192e+00  1.434e+00]
        cost: 1.4334228921820125
         jac: [[-4.421e-03  0.000e+00 ... -4.586e-06 -1.134e-03]
               [-2.239e-05  0.000e+00 ... -2.385e-03 -2.388e-03]
               ...
               [ 1.152e-02  0.000e+00 ...  3.051e-02  1.616e-02]
              

# Step 5: Run MCMC
There are many different options for the `run_mcmc` function. It operates using an `emcee` `EnsembleSampler`. All of the different options are shown in the docstring. Not that the interactive plotting only works in a `.py` script and not a notebook. Always set `include_sums` to be false. The modeled double sums are nonsense for CHARIS. There is an auto convergence checker that will stop the simulation when it has detected convergence, however this does not always work super well and often takes way too long. While MCMC is running, you can feel free to plot the output h5 file or summarize posteriors. There are functions in `plotting.py` for this. You can see how to use them in `development_notebooks/MCMC_plots.ipynb`.


In [ ]:
ndim = 6  # Number of parameters to fit
pool_processes = 12 # number of CPU cores to use
nwalkers = max(2 * ndim, pool_processes * 2) # number of walkers at least twice the number of dimensions
if nwalkers % pool_processes != 0:
    nwalkers += pool_processes - (nwalkers % pool_processes)

print(f"{nwalkers} walkers for {ndim} parameters")
sampler, p_keys = run_mcmc(p0_dict, system_mm, interleaved_values_all,configuration_list_all,prior_dict,bounds,output_h5,nwalkers=nwalkers,pool_processes=pool_processes,process_model=process_model, process_dataset=process_dataset,nsteps=40000, include_sums=False)

Auto-detected: 12 processes, 24 walkers for 6 parameters


<IPython.core.display.Javascript object>

 21%|██        | 8257/40000 [1:40:57<6:28:05,  1.36it/s]


emcee: Exception while calling your likelihood function:emcee: Exception while calling your likelihood function:emcee: Exception while calling your likelihood function:emcee: Exception while calling your likelihood function:emcee: Exception while calling your likelihood function:emcee: Exception while calling your likelihood function:emcee: Exception while calling your likelihood function:
emcee: Exception while calling your likelihood function:



  params:

emcee: Exception while calling your likelihood function:  params:emcee: Exception while calling your likelihood function:  params:  params:  params:  params: 
 
  params:emcee: Exception while calling your likelihood function: 
    params:    params:emcee: Exception while calling your likelihood function:  params:
     params:
   params: [ 8.97358762e-01  1.17657652e+00  9.47171586e-01 -2.66086563e-01
 -5.74666090e+00  9.67671029e+89][ 5.52101160e-01  1.56788460e+00  1.22836809e+00 -2.76688753e+00
  2.44808533e+00 -1.59618238e+90]

KeyboardInterrupt: 